In [1]:
# Set up price and volume for aFFR opp
from entsoe import EntsoePandasClient
import pandas as pd

client = EntsoePandasClient(api_key='c008ebef-0f0f-42b2-a2ae-7330192177ac')

start = pd.Timestamp('20230101', tz='Europe/Brussels')
end = pd.Timestamp('20231231', tz='Europe/Brussels')
country_code = 'NO_1'  # Belgium
type_marketagreement_type = 'A01'

# get the data
day_ahead = client.query_day_ahead_prices(country_code, start=start, end=end) #KeyError: '60T'
reservemarked_pris = client.query_contracted_reserve_prices(country_code, type_marketagreement_type, start=start, end=end, psr_type=None)
reservemarked_volum = client.query_contracted_reserve_amount(country_code, type_marketagreement_type, start=start, end=end, psr_type=None)

#clean the data
reservemarked_pris.columns = reservemarked_pris.columns.droplevel(0) #remove multiindex in columns
reservemarked_pris = reservemarked_pris.drop(columns=['Symmetric']) #del 'Frequency containment reserve' column
reservemarked_pris = reservemarked_pris.rename(columns={'Down': 'aFRR down EUR/MW'}) #rename columnns
reservemarked_pris = reservemarked_pris.rename(columns={'Up': 'aFRR up EUR/MW'}) #rename columnns

reservemarked_volum.columns = reservemarked_volum.columns.droplevel(0) #remove multiindex in columns
reservemarked_volum = reservemarked_volum.drop(columns=['Symmetric']) #del 'Frequency containment reserve' column
reservemarked_volum = reservemarked_volum.rename(columns={'Down': 'aFRR down MW'}) #rename columnns
reservemarked_volum = reservemarked_volum.rename(columns={'Up': 'aFRR up MW'}) #rename columnns


reservemarked_pris = reservemarked_pris.reset_index() #remove index and set it as a column
reservemarked_pris = reservemarked_pris.rename(columns={'index': 'Datetime'}) #change name of column
reservemarked_pris['Datetime'] = reservemarked_pris['Datetime'].dt.tz_convert('UTC') #convert to UTC

reservemarked_volum = reservemarked_volum.reset_index() #remove index and set it as a column
reservemarked_volum = reservemarked_volum.rename(columns={'index': 'Datetime'}) #change name of column
reservemarked_volum['Datetime'] = reservemarked_volum['Datetime'].dt.tz_convert('UTC') #convert to UTC


#merge the data into areas['NO1']
areas['NO1'] = pd.merge(areas['NO1'], reservemarked_pris, how='left', right_on="Datetime", left_on="Datetime")
areas['NO1'] = pd.merge(areas['NO1'], reservemarked_volum, how='left', right_on="Datetime", left_on="Datetime")


/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level = [0,1]).mean()
/Users/mathiasmollatt/opt/miniconda3/envs/inf200june/lib/python3.11/site-packages/entsoe/parsers.py:449: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df

NameError: name 'areas' is not defined